### Train for MICCAI challenge on colab using data on gDrive

In [ ]:
# setup
!apt-get update

In [2]:
# imports
from ails_miccai_uwf4dr_challenge.dataset import DatasetBuilder
# Pytorch
import torch
import torch.nn as nn

In [ ]:
# connect to gDrive
use_google_drive = True
if use_google_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    ails_data_base_path = "/content/drive/shared-with-me/data"
    my_data_base_path = "/content/drive/My Drive/JKU/AILS_CHALLENGE_2024"

In [ ]:
# select device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))

In [ ]:
# setup dataset
original_data_dir = ails_data_base_path + "/raw"
external_data_dir = ails_data_base_path + "/external"
dataset_builder = DatasetBuilder(dataset='all', task='task1', original_data_dir=original_data_dir, external_data_dir=external_data_dir).get_train_val()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Dataset length:  508
Dataset length:  128


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: moritsih. Use `wandb login --relogin` to force relogin


/Users/moritz/miniconda3/envs/AILS-MICCAI-UWF4DR-Challenge/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /Users/moritz/Documents/Master/AILS-MICCAI-UWF4DR-Challenge/models/checkpoints exists and is not empty.

  | Name           | Type                   | Params | Mode 
------------------------------------------------------------------
0 | model          | ResNet                 | 11.2 M | train
1 | loss_fn        | BCEWithLogitsLoss      | 0      | train
2 | train_accuracy | BinaryAccuracy         | 0      | train
3 | val_accuracy   | BinaryAccuracy         | 0      | train
4 | test_accuracy  | BinaryAccuracy         | 0      | train
5 | train_mcc      | BinaryMatthewsCorrCoef | 0      | train
6 | val_mcc        | BinaryMatthewsCorrCoef | 0      | train
7 | test_mcc       | BinaryMatthewsCorrCoef | 0      | train
8 | val_auroc      | BinaryAUROC            | 0      | train
9 | test_auroc     | BinaryAUROC            | 0      | t

Epoch 1: 100%|██████████| 64/64 [02:15<00:00,  0.47it/s, v_num=i69z]       

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 64/64 [02:15<00:00,  0.47it/s, v_num=i69z]


<function wandb.sdk.wandb_run.finish(exit_code: Optional[int] = None, quiet: Optional[bool] = None) -> None>